<a href="https://colab.research.google.com/github/mhieule/.github/blob/main/FantasyPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import all necessary libraries

In [1]:
import requests
import json
import pandas as pd
import numpy as np
from pprint import pprint

# Make a GET request from the API endpoint

In [2]:
"""
Base url
"""

base_url = 'https://fantasy.premierleague.com/api/'
url = base_url+'bootstrap-static/'

"""
Perform GET request
"""

r_json = requests.get(url).json()
pprint(r_json, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 8723192}


# Find out the important JSON keys aka "elements", "element_type" and "teams"
TODO: Expand the model by finding possible important keys among the rest

In [3]:
r_json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

# Build DataFrame from the API request response

In [4]:
elements_df = pd.DataFrame(r_json['elements'])
elements_types_df = pd.DataFrame(r_json['element_types'])
teams_df = pd.DataFrame(r_json['teams'])

## Players DataFrame

In [5]:
print(elements_df.columns)
elements_df.head()

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_con

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,100.0,100.0,232223,0,0,0,0,0,4,1.5,...,436,68,460,43,460,43,199,38,0.0,0.0
1,NaN,NaN,58822,0,0,0,0,0,2,1.5,...,554,154,289,90,289,90,251,92,0.0,0.0
2,0.0,0.0,153256,0,0,0,0,0,3,0.0,...,362,206,417,145,417,145,439,150,0.0,0.0
3,NaN,NaN,438098,0,0,0,0,0,3,2.6,...,137,89,634,270,634,270,416,136,0.0,0.0
4,NaN,NaN,226597,0,0,0,0,0,2,2.8,...,255,31,242,86,242,86,7,3,0.0,0.0


### Lite Players DataFrame

In [6]:
# Handpick the important attributes of players
lite_players_df = elements_df[['second_name','first_name','team','element_type','form','selected_by_percent','now_cost','total_points','value_season']]
lite_players_df.head()

# Map players to their position and change the column name to 'position'
lite_players_df['element_type'] = lite_players_df.element_type.map(elements_types_df.set_index('id').singular_name)
lite_players_df.rename(columns={'element_type': 'position'})
lite_players_df.head()

# Map players to their team
lite_players_df['team'] = lite_players_df.team.map(teams_df.set_index('id').name)
lite_players_df.head(5)

<ipython-input-6-9f4d85aedfcb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lite_players_df['element_type'] = lite_players_df.element_type.map(elements_types_df.set_index('id').singular_name)
<ipython-input-6-9f4d85aedfcb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lite_players_df['team'] = lite_players_df.team.map(teams_df.set_index('id').name)


,second_name,first_name,team,element_type,form,selected_by_percent,now_cost,total_points,value_season
0,Balogun,Folarin,Arsenal,Forward,0.0,0.7,45,0,0.0
1,Alves Soares,Cédric,Arsenal,Defender,0.0,0.4,40,0,0.0
2,Elneny,Mohamed,Arsenal,Midfielder,0.0,0.1,45,0,0.0
3,Ferreira Vieira,Fábio,Arsenal,Midfielder,0.0,0.1,55,0,0.0
4,dos Santos Magalhães,Gabriel,Arsenal,Defender,1.0,31.4,50,1,0.2


## Teams DataFrame

In [7]:
print(teams_df.columns)
teams_df.head()

Index(['code', 'draw', 'form', 'id', 'loss', 'name', 'played', 'points',
       'position', 'short_name', 'strength', 'team_division', 'unavailable',
       'win', 'strength_overall_home', 'strength_overall_away',
       'strength_attack_home', 'strength_attack_away', 'strength_defence_home',
       'strength_defence_away', 'pulse_id'],
      dtype='object')


,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1230,1285,1250,1250,1210,1320,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1115,1175,1130,1190,1100,1160,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,None,False,0,1060,1095,1050,1100,1060,1090,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,None,False,0,1125,1205,1120,1220,1130,1190,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,None,False,0,1165,1210,1120,1200,1210,1240,131


# Exploratory Analysis on Players Data

Here we will extract the most significant statistics on players data which was preprocessed in the last step. For the time being, there will only be most valueable players, positions and teams.

**TODO**: Advanced filters and sortings on multiple performance indices. \\
**TODO**: Plot the correlation between ... and point \\
**TODO**: Plot the performances of players to compare \\

## Most valueable players

In [8]:
# Use astype() to convert the values in the column "value_season" to be floats
lite_players_df['value_season'] = lite_players_df.value_season.astype(float)

# Use sort_values() to sort the values of the players
lite_players_df.sort_values('value_season', ascending=False).head(10)

<ipython-input-8-a4c92970b8ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lite_players_df['value_season'] = lite_players_df.value_season.astype(float)


,second_name,first_name,team,element_type,form,selected_by_percent,now_cost,total_points,value_season
449,Varane,Raphaël,Man Utd,Defender,14.0,2.3,50,14,2.8
308,Leno,Bernd,Fulham,Goalkeeper,12.0,8.2,45,12,2.7
455,Wan-Bissaka,Aaron,Man Utd,Defender,12.0,1.8,45,12,2.7
418,Hernandez,Rodrigo,Man City,Midfielder,13.0,5.8,55,13,2.4
238,Disasi,Axel,Chelsea,Defender,11.0,1.6,50,11,2.2
243,Andersen,Joachim,Crystal Palace,Defender,9.0,1.9,45,9,2.0
456,Onana,André,Man Utd,Goalkeeper,9.0,28.1,50,9,1.8
574,Leite de Souza Junior,Emerson,Spurs,Defender,8.0,1.1,45,8,1.8
257,Mitchell,Tyrick,Crystal Palace,Defender,8.0,0.6,45,8,1.8
304,Diop,Issa,Fulham,Defender,8.0,0.8,45,8,1.8


## Most valueable positions

In [14]:
# Create a copy of lite_players_df in which the players that have value <= 0 are removed
positive_lite_players_df = lite_players_df.loc[lite_players_df.value_season > 0]

# Pivot table to aggregate after position
pivot = positive_lite_players_df.pivot_table(index='element_type', values = 'value_season', aggfunc = np.mean).reset_index()
pivot.sort_values('value_season', ascending=False)



,element_type,value_season
2,Goalkeeper,0.720000
0,Defender,0.605618
1,Forward,0.591892
3,Midfielder,0.438235


## Most valueable teams

In [19]:
# Pivot table to aggregate after team
pivot_team = positive_lite_players_df.pivot_table(index=['team'], values = 'value_season', aggfunc = np.mean).reset_index()
pivot_team.sort_values('value_season', ascending=False)

,team,value_season
7,Crystal Palace,1.050000
9,Fulham,0.969231
12,Man City,0.793750
13,Man Utd,0.780000
4,Brighton,0.662500
14,Newcastle,0.656250
17,Spurs,0.538462
0,Arsenal,0.515385
11,Luton,0.490909
6,Chelsea,0.473333


## Most valueable player within teams

In [24]:
# Pivot table to aggregate after team to find out the max value player within each team
pivot_team_max = positive_lite_players_df.pivot_table(index=['team'], values = 'value_season', aggfunc = np.max).reset_index()
pivot_team_max.sort_values('value_season', ascending=False)

#Merge with the original dataframe to find the names and other stats of the Team MVPs
team_mvp_df = pivot_team_max.merge(lite_players_df, how="left", on=['team', 'value_season'])
team_mvp_df.sort_values('value_season', ascending=False)

,team,value_season,second_name,first_name,element_type,form,selected_by_percent,now_cost,total_points
29,Man Utd,2.8,Varane,Raphaël,Defender,14.0,2.3,50,14
25,Fulham,2.7,Leno,Bernd,Goalkeeper,12.0,8.2,45,12
28,Man City,2.4,Hernandez,Rodrigo,Midfielder,13.0,5.8,55,13
12,Chelsea,2.2,Disasi,Axel,Defender,11.0,1.6,50,11
13,Crystal Palace,2.0,Andersen,Joachim,Defender,9.0,1.9,45,9
34,Spurs,1.8,Leite de Souza Junior,Emerson,Defender,8.0,1.1,45,8
31,Newcastle,1.7,Barnes,Harvey,Midfielder,11.0,3.6,65,11
30,Newcastle,1.7,Isak,Alexander,Forward,13.0,21.6,76,13
0,Arsenal,1.5,Nketiah,Eddie,Forward,8.0,4.1,55,8
3,Brentford,1.5,Wissa,Yoane,Forward,9.0,5.5,60,9
